# Imports

In [1]:
import pandas as pd
import os

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline

Big Data Charakterstics
1. Volume: Tweets sind in der Regel kurze Nachrichten mit begrenztem Umfang, aber die Anzahl der Tweets, die jeden Tag gepostet werden, ist enorm. Laut Twitter selbst generieren Nutzer mehr als 500 Millionen Tweets pro Tag. Diese große Menge an Daten erfordert eine effektive Verarbeitungstechnologie, um die Daten zu speichern, zu verarbeiten und zu analysieren.

Text ohne Metadaten (-> reiner Text):
- 1 Tweet hat also im Schnitt: 28 * 8 Bits = 224 Bits = 28 Byte
- hochgerechnet auf die Zahlen von oben bedeutet das also:
- 6.000 Tweets pro Sekunde      --> 168.000 Byte = 168 Kilobyte pro Tag
- = 360.000 Tweets pro Minute    --> 10.080.000 Byte = 10,08 Megabyte pro Tag
- = 21.600.000 Tweets pro Stunde     --> 604.800.000 Byte = 604,8 Megabyte pro Tag
- = 518.400.000 Tweets pro Tag    --> 14.515.200.000 Byte = 14,51 Gigabyte pro Tag



Text mit Metadaten und ohne Bilder/Videos:
- 2656 Tweets --> 14.673.664 Bytes --> 5.524 Byte pro Tweet
- 1 Tweet hat also im Schnitt inkl. Metadaten: 5.524 Byte
- hochgerechnet auf die Zahlen von oben bedeutet das also:
- 6.000 Tweets pro Sekunde --> 33.144.000 Byte = 33,144 Megabyte pro Tag
- = 360.000 Tweets pro Minute    --> 1.988.640.000 Byte = 1,988 Gigabyte pro Tag
- = 21.600.000 Tweets pro Stunde     --> 119.318.400.000 Byte = 119,3 Gigabyte pro Tag
- = 518.400.000 Tweets pro Tag    --> 2.863.641.600.000 Byte = 2,86 Terabyte pro Tag

2. Variety: Tweets enthalten verschiedene Arten von Daten wie Text, Bilder, Videos, Links und Metadaten. Diese Vielfalt an Daten erfordert eine Technologie, die in der Lage ist, verschiedene Datentypen zu verarbeiten und zu analysieren.

3. Velocity: Tweets werden in Echtzeit gepostet, was bedeutet, dass die Datenströme schnell und kontinuierlich sind.  
Maximale Anzahl an Tweets die jemals gemessen wurde: 618 725 Tweets/min

Warum kann das Problem nicht mit herkömmlichen Storage/Analyse/Datenbank-technologien gelöst werden?
- Hate Speech-Analyse ist mit herkömmlichen mitteln möglich, jedoch nicht in dem Volumen und Geschwindigkeit.
- Der Trainingsdatensatz sollte ständig erweitert und neue Modelle trainiert werden -> stetig komplexere Analysen und Trainingsdurchläufe.

Setup/Bootstrapping Kosten
- Hardware-Kosten: Speicher, CPU, Rack ...
- Potenzielle Integrationskosten: Anbindung an vorhandenen Systeme von Twitter

Skew und Bias der Daten:
- Daten sind in in Realität deutlich Biased (eine viel größere Anzahl an Non-Hate-Speech liegt vor). In unserem Trainingsdatensatz sind die Labels etwa gleichverteilt. Die Tweets können somit ohne großen skew zwischen den Knoten verteilt werden.


# Prepare Training Data

Setup

In [2]:
import findspark

findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, regexp_replace, lower
from pyspark.ml import Pipeline
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, CountVectorizer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator


MAX_MEMORY = "6g"
spark = SparkSession.builder \
                    .appName('multi_class_text_classifier')\
                    .master("local[8]") \
                    .config("spark.executor.memory", MAX_MEMORY) \
                    .config("spark.driver.memory", MAX_MEMORY) \
                    .getOrCreate()

print("Apache Spark version: ", spark.version)
spark

Apache Spark version:  3.3.1


Trainingsdaten aus dem Data Lake (Parquet File) laden

In [3]:
spark_df = spark.read.parquet('../data/parquet_data')
spark_df = spark_df.withColumnRenamed("tweet_text","text")
spark_df.count()

1198584

# Data Preparation
Methode für die Bereinigung der Texte (z.B. Groß- und Kleinschreibung, Sondernzeichen, ...)

In [5]:
def clean_text(c):
  c = lower(c)
  c = regexp_replace(c, "(https?\://)\S+", "") # Remove links
  c = regexp_replace(c, "(\\n)|\n|\r|\t", "") # Remove CR, tab, and LR
  c = regexp_replace(c, "(?:(?:[0-9]{2}[:\/,]){2}[0-9]{2,4})", "") # Remove dates
  c = regexp_replace(c, "@([A-Za-z0-9_]+)", "") # Remove usernames
  c = regexp_replace(c, "[0-9]", "") # Remove numbers
  c = regexp_replace(c, "\:|\/|\#|\.|\?|\!|\&|\"|\,", "") # Remove symbols
  return c

spark_df = spark_df.withColumn("text", clean_text(col("text")))

spark_df.show(5)

+-------------------+--------------------+--------------+
|              index|                text|majority_label|
+-------------------+--------------------+--------------+
|1108866829991272448|@ finna fuck pont...|             0|
|1058874314303320064|t don mind me ’ i...|             1|
|1109486326477438976|a law played jude...|             0|
|1062399239337140224|review of heart b...|             0|
|1113926202006360064|nigga when the yo...|             0|
+-------------------+--------------------+--------------+
only showing top 5 rows



Den gesamten Datensatz oder einen Anteil des Datensatzes für das Training auswählen.

In [16]:
spark_df_sample = spark_df#.sample() #fraction=0.1
spark_df_sample.count()

1198584

# Feature Engineering
Für das Features Engineering wurden zwei Methoden für die Umwandlung der natürlichsprachlichen Texte in strukturierte Daten untersucht.
- Tokenization und Word2Vec (Embedding): Basierend auf den einzelnen Wörtern in einem Tweet, wird für jeden Satz ein Vektor mit 300 Dimensionen gebildet. 
- Tokenization und CountVectorizer: Nach der Aufteilung der Tweets in einzelne Wörter, werden die 10000 häufigsten Wörter in jedem Tweet gezählt. Es entsteht ein Vektor mit 10000 Dimensionen.

Die beste Leistung konnte mit dem CountVectorizer erzielt werden. 
Weitere Optionen, wie z.B. TF-IDF oder Embeddings von großen Sprachmodellen wurden nicht untersucht. 

## Tokenization und Word2Vec

In [6]:
# from pyspark.ml.feature import Word2Vec
# from pyspark.ml import Pipeline
# from pyspark.ml.feature import Tokenizer
# from pyspark.ml.feature import StopWordsRemover

# # 'We hate religion' > 'We' 'hate' 'religion'
# tokenizer = Tokenizer(inputCol="text", outputCol="tokens")

# # 'We' > (0.000, 0.032432, ...) 300 Dimensionen
# w2v = Word2Vec(vectorSize=300, minCount=0, inputCol="tokens", outputCol="features")

# doc2vec_pipeline = Pipeline(stages=[tokenizer, w2v])
# doc2vec_model = doc2vec_pipeline.fit(spark_df_sample)
# doc2vecs_df = doc2vec_model.transform(spark_df_sample)

# doc2vec_model.write().overwrite().save("../models/prep_tok2vec")

## Tokenization und CountVectorizer

In [7]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, CountVectorizer


# 'We hate religion' > 'We' 'hate' 'religion'
regexTokenizer = RegexTokenizer(inputCol="text", outputCol="tokens", pattern="\\W")

# Remove stop words
stopwordsRemover = StopWordsRemover(inputCol="tokens", outputCol="filtered")

# Term frequency
countVectors = CountVectorizer(inputCol="filtered", outputCol="features", vocabSize=10000, minDF=5)

doc2tf_pipeline = Pipeline(stages=[regexTokenizer, stopwordsRemover, countVectors])
doc2tf_model = doc2tf_pipeline.fit(spark_df_sample)
doc2tf_df = doc2tf_model.transform(spark_df_sample)

In [8]:
doc2tf_model.write().overwrite().save("../models/prep_tok2tf")

# Model Training
Für das Training wurden die klassischen ML-Modelle Logistic Regression und Naive Bayes untersucht. Für das weitere Vorgehen wurde das Naive Bayes-Modell ausgewählt.

In [9]:
# Data from Tok2Tf
hate_train_df, hate_test_df = doc2tf_df.randomSplit([0.8, 0.2])

In [10]:
print("Training Dataset Count: " + str(hate_train_df.count()))
times_hate = hate_train_df.filter(hate_train_df['majority_label'] > 0.0).count()
print(f'Times hate in training: {times_hate}')
times_not_hate = hate_train_df.filter(hate_train_df['majority_label'] == 0.0).count()
print(f'Times not hate in training: {times_not_hate}')

print("Test Dataset Count: " + str(hate_test_df.count()))
times_hate = hate_test_df.filter(hate_test_df['majority_label'] > 0.0).count()
print(f'Times hate in test: {times_hate}')
times_not_hate = hate_test_df.filter(hate_test_df['majority_label'] == 0.0).count()
print(f'Times not hate in test: {times_not_hate}')

Training Dataset Count: 958700
Times hate in training: 236164
Times not hate in training: 722536
Test Dataset Count: 239884
Times hate in test: 59604
Times not hate in test: 180280


## Logistic Regression

In [11]:
# from pyspark.ml.classification import LogisticRegression

# lr_classifier = LogisticRegression(family="multinomial", labelCol="majority_label", featuresCol="features")

# lr_classifier_pipeline = Pipeline(stages=[lr_classifier])
# lr_trained_pipeline = lr_classifier_pipeline.fit(hate_train_df)
# predictions = lr_trained_pipeline.transform(hate_test_df)

# lr_model_evaluator = MulticlassClassificationEvaluator(
#     labelCol="majority_label", predictionCol="prediction", metricName="accuracy")

# accuracy = lr_model_evaluator.evaluate(predictions)
# print("Accuracy = %g" % (accuracy))

# times_hate = predictions.filter(predictions['prediction'] == 1.0).count()
# print(f'Times hate detected: {times_hate}')
# times_not_hate = predictions.filter(predictions['prediction'] == 0.0).count()
# print(f'Times not hate detected: {times_not_hate}')

Save and Load Logistic Regression

In [12]:
# lr_trained_pipeline.write().overwrite().save("../models/model_lr")

## Naive Bayes

In [13]:
from pyspark.ml.classification import NaiveBayes
classifier = NaiveBayes(smoothing=1, labelCol="majority_label", featuresCol="features")

classifier_pipeline = Pipeline(stages=[classifier])
predictions = classifier_pipeline.fit(hate_train_df).transform(hate_test_df)

Evaluation

In [14]:
model_evaluator = MulticlassClassificationEvaluator(
    labelCol="majority_label", predictionCol="prediction", metricName="accuracy")

accuracy = model_evaluator.evaluate(predictions)
print("Accuracy = %g" % (accuracy))

times_hate = predictions.filter(predictions['prediction'] == 1.0).count()
print(f'Times hate detected: {times_hate}')
times_not_hate = predictions.filter(predictions['prediction'] == 0.0).count()
print(f'Times not hate detected: {times_not_hate}')

Accuracy = 0.766279
Times hate detected: 50350
Times not hate detected: 189534


Save and Load Naive Bayes

In [15]:
classifier_pipeline.write().overwrite().save("../models/model_nb")

# Training Benchmark

Den gesamten Datensatz oder einen Anteil des Datensatzes für das Training auswählen. Je größer der Trainingsdatensatz, desto länger benötigten die Pipeline für die Ausführung. Für die  Ausführung ausgewählt wurden drei Datensätze kuriert. Weitere Benchmarks zu den Datensätzen sind in dem Notebook compare_data_storage_formats zu finden.

In dem nachstehenden Notebook ist die Implementierung in MLLib und eine Vergleichsimplementierung in Sklearn abgebildet. Den Source-Code für die Ausführung des Benchmark mit varierenden Kernanzahlen für die MLLib-Implementierung findet sich in der Datei: "Benchmark_Training_Pipeline_Spark.py"
Die erhobenen Messergebnisse finden sich in dem Ordner Benchmark-Ergebnisse.

Für jede Konfiguration wurden 5 Testabläufe durchgeführt und der Mittelwert sowie die Standardabweichung berechnet.

Benchmark der Trainingszeiten MLlib:
| Anzahl Trainingsdatensätze | Mean 1 Kern | Mean 2 Kerne | Mean 4 Kerne | Mean 8 Kerne |
| --- | --- | --- | --- | --- |
| small | 5.04s (0.21s) | 3.32s (0.19s) | 2.47s (0.31s) | 2.42s (0.23s) |
| medium | 17.35s (0.44s) | 10.25s (0.08s) | 6.44s (0.10s) | 5.86s (0.17s) |
| complete | 31.85s (0.75s) | 18.88s (0.36s) | 12.47s (0.33s) | 11.25s (0.45s) |

Benchmark der Trainingszeiten Sklearn:
| Anzahl Trainingsdatensätze | Mean |
| --- | --- |
| 120461 | 2.96s (0.31s) |
| 599259 | 11.88s (0.44s) |
| 1198584 | 22.56s (0.43s) |

Ergebnisse:
- In unserem Test liefert Spark MLLib bei kleinen Datenmengen und wenigen Kernen minimal schlechtere Ergebnisse. Dies könnte z.B. durch Computational Overhead für die Spark Infrastruktur sein. Allerdings ist die Abweichung sehr gering.
- In unserem Test skaliert die Spark MLLib Trainingspipeline sehr gut mit steigender Anzahl der lokal zur Verfügung gestellten Kerne. Diese deutet auf eine gute horizontale Skalierbarkeit hin.
- In unserem Test skaliert die Spark MLLib Trainingspipeline bei größer werdenen Datenmengen besser. Dies könnte durch die verteilte Berechnung in der Spark Infrastruktur erreicht werden.  

### Anmerkungen zur Durchführung des Benchmarks: 
Local Spark Environment (3.3.1)
- 4 Cores, 8 Threats (1.8 Ghz Base Clock)
- 8 GB RAM (shared with OS)

Einschränkungen:
- Trotz größter Sorgfalt können beeinflussende Faktoren bei der manuellen Durchführung der Tests (z.B. durch Hintergrundprozsse) nicht ausgeschlossen werden. 
- Die Ergebnisse des Benchmark sind stark abhängig von der Implementierung in den Frameworks.  
- Für das schnelle Prototyping wurde eine lokale Spark-Installation gewählt. Diese kann maximal über die Ressourcen des Host-PCs verfügen. 
- Die Leistungssteigerungen von 8 auf 4 Kerne fallen erwartungsgemäß nicht sehr groß aus, da hier nicht mehr physische Kerne als vorher genutzt werden können.

## MLlib implementation

In [4]:
from pyspark.ml.feature import CountVectorizer
from pyspark.ml import Pipeline

spark_df_sample = spark_df.sample(fraction=0.1) 
spark_df_sample.count()

598566

In [5]:
hate_train_df, hate_test_df = spark_df_sample.randomSplit([0.8, 0.2])

In [6]:
# 'We hate religion' > 'We' 'hate' 'religion'
regexTokenizer = RegexTokenizer(inputCol="text", outputCol="tokens", pattern="\\W")

# Remove stop words
stopwordsRemover = StopWordsRemover(inputCol="tokens", outputCol="filtered")

# Term frequency
countVectors = CountVectorizer(inputCol="filtered", outputCol="features", vocabSize=10000, minDF=5)

# Classifier
classifier = NaiveBayes(smoothing=1, labelCol="majority_label", featuresCol="features")

In [9]:
inference_pipeline = Pipeline(stages=[regexTokenizer, stopwordsRemover, countVectors, classifier])
trained_inference_pipeline = inference_pipeline.fit(hate_train_df)
predictions = trained_inference_pipeline.transform(hate_test_df)

Save MLlib Pipeline

In [21]:
trained_inference_pipeline.write().overwrite().save("../models/mllib_model_nb")

In [22]:
model_evaluator = MulticlassClassificationEvaluator(
    labelCol="majority_label", predictionCol="prediction", metricName="accuracy")

accuracy = model_evaluator.evaluate(predictions)
print("Accuracy = %g" % (accuracy))

Accuracy = 0.767052


## SKlearn implementation

In [23]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline

pandas_df = spark_df.toPandas()
pandas_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1198584 entries, 0 to 1198583
Data columns (total 3 columns):
 #   Column          Non-Null Count    Dtype 
---  ------          --------------    ----- 
 0   index           1198584 non-null  int64 
 1   text            1198584 non-null  object
 2   majority_label  1198584 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 27.4+ MB


In [24]:
pandas_df_sample = pandas_df.sample(n=599259)
pandas_df_sample.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 599259 entries, 1106893 to 314064
Data columns (total 3 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   index           599259 non-null  int64 
 1   text            599259 non-null  object
 2   majority_label  599259 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 18.3+ MB


In [25]:
X_train, X_test, y_train, y_test = train_test_split(pandas_df_sample['text'], pandas_df_sample['majority_label'], train_size = 0.8, test_size=0.2)

In [26]:
#120461: 2.9s, 2.8s, 2.9s, 3.5s, 2.7s
#599259: 11.4s, 11.8s, 11.7s, 11.9s, 12.6s
#1198584: 22.1s, 22.4s, 22.8s, 22.3s, 23.2s

In [27]:
countVec = CountVectorizer(stop_words='english', min_df=2, max_features=10000)
clf = MultinomialNB()

In [28]:
inference_pipeline = Pipeline([
    ('vect', countVec),
    ('clf', clf),
])

trained_inference_pipeline = inference_pipeline.fit(X_train, y_train)
y_pred = trained_inference_pipeline.predict(X_test)

Save Sklearn Pipeline

In [29]:
import joblib
joblib.dump(trained_inference_pipeline, '../models/sklearn_model_nb.pkl')
# pipeline = joblib.load('pipeline.pkl')

['../models/sklearn_model_nb.pkl']

In [30]:
accuracy_score(y_test, y_pred)

0.7686563428228148

In [31]:
from statistics import mean, stdev
small_sklearn = [2.9, 2.8, 2.9, 3.5, 2.7]
print(f"Small Sklearn mean: {mean(small_sklearn)} stdev: {stdev(small_sklearn)}")
medium_sklearn = [11.4, 11.8, 11.7, 11.9, 12.6]
print(f"Medium Sklearn mean: {mean(medium_sklearn)} stdev: {stdev(medium_sklearn)}")
complete_sklearn = [22.1, 22.4, 22.8, 22.3, 23.2]
print(f"Complete Sklearn mean: {mean(complete_sklearn)} stdev: {stdev(complete_sklearn)}")

Small MLLib mean: 3.3000000000000003 stdev: 0.4242640687119285
Medium MLLib mean: 8.98 stdev: 0.6300793600809349
Complete MLLib mean: 17.4 stdev: 1.0368220676663862
Small Sklearn mean: 2.96 stdev: 0.31304951684997057
Medium Sklearn mean: 11.88 stdev: 0.44384682042344276
Complete Sklearn mean: 22.56 stdev: 0.4393176527297754
